# **Regression**
Imagine that you work for SL, and that one of your customers' biggest complaints is that they don't know what's the arrival delay for their buses. You decide to do something about it by building a model to predict the arrival delay at one bus stop. You intend to use the massive amounts of delay data the company has collected over the years to build a machine-learning model.

Regression models predict numeric outcomes such as the price of a car, the age of a person, or the delay at a bus stop. Let's use a portion of a larger bus-delay dataset from Stockholm to train a regression model to predict the arrival delay given the dwell time, scheduled travel time, upstream stop delay, and recurrent delay.

# **Tutorial 1: The development process of an AI model (linear regression)**
In tutorial 1, we will give an example of a simple linear regression model to illustrate the process of developing an AI model which includes data exploration​, feature engineering, linear regression model specification​, model training​, and cross-validation.​



## Load and prepare the data
Start by loading the dataset and shaping it so that it's suitable for use in machine learning. This dataset is a subset of a much larger dataset. The data requires an amount of prep work before it's of any use at all. We here use a url link to load the data. If you cannot load the data via URL link, please download the data from GitHub and upload the data to colab manually.

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/zhenliangma/Applied-AI-in-Transportation/master/Exercise_2_regression_model/Exercise2BusData.csv'
df = pd.read_csv(url)

# df = pd.read_csv('Exercise2BusData.csv')
df.head(10)

How many rows and columns does the dataset contain?

In [ ]:
df.shape

Are any of the columns missing values?

In [ ]:
df.info()

For simplicity, we only use 1000 records here. If your computer cannot run this tutorial in a short time, please set a smaller number in the following code. For example, 800.

In [ ]:
df = df.iloc[:1000]

Remove columns that are no longer needed.

In [ ]:
df = df.drop(['Arrival_time','Stop_id','Bus_id','Line_id'], axis=1)

Draw a histogram by using seaborn to show the distribution of arrival delay.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

sns.histplot(x=df['Arrival_delay'])

Find out how much influence input variables such as "Upstream_stop_delay" and "Dwell_time" have on the values in the "Arrival_delay" column.

In [ ]:
corr_matrix = df.corr()
corr_matrix['Arrival_delay'].sort_values(ascending=False)

Feature engineering visually explore and understand the relationships between different features (variables) in a dataset. Seaborn pair plots help identify patterns, correlations, and potential outliers, aiding in the selection and transformation of features to improve model performance. It is a crucial step in data preprocessing to enhance the accuracy and effectiveness of machine learning models.​

In [ ]:
x = df.drop(['Arrival_delay'], axis=1)  #drop the y; think about mse etc
y = df['Arrival_delay']

sns.pairplot(x)

## Train / Test Split

Before modeling we need to split our data into a training and test set. The training set is used to train the model and the test set to evaluate its performance.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) #why set randomstate= 42

## Create a linear regression model
Now it's time build a regression model and train it with the data prepared in the previous exercise.

In [ ]:
from sklearn.linear_model import LinearRegression # easy model first

# Create a Linear Regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train, y_train)  # didnt specify parameter tuning; used default

# Predict the test data with the fitted model
y_pred = model.predict(X_test)

In [ ]:
#xgboost
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
params = {
    "booster": "gblinear",         # Use linear base learner
    "objective": "reg:squarederror"  # Regression objective
}
xgb_model = xgb.train(params=params, dtrain=dtrain, num_boost_round=100)
preds = xgb_model.predict(dtest)



## Evaluate the model
You can evaluate the model's performance using various metrics, such as Mean Absolute Error (MAE), Mean Squared Error (MSE), or R-squared (R2).

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

## Visualize the Results
we create a "Actual vs. Predicted Values" graph to give a visual inspection of the prediction quality.

In [ ]:
import matplotlib.pyplot as plt

# y_test contains the actual target values for the test dataset
# y_pred contains the predicted values for the test dataset

# Create a scatter plot to visualize the relationship
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)  # Plot actual vs. predicted values

# Add labels and title
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values")

# Add a diagonal line for reference (perfect predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', lw=2)

# Show the plot
plt.show()

# **Tutorial 2: AI model training techniques (SVM)**
We will give a SVM model to illustrate how to use training techniques including train-test split, normalization, and grid search to find the best hyperparameters for the model.

## Train-test split
The purpose of a "train-test split" is to evaluate the performance of a machine learning model. It involves dividing a dataset into two subsets: the training set, used to train the model, and the test set, used to assess how well the model generalizes to new, unseen data. This process helps identify if the model is overfitting (performing well on the training data but poorly on new data) or underfitting (performing poorly on both training and test data), allowing for adjustments to improve model accuracy and generalization.

We have already split the data in Tutorial 1. Therefore, we just put the code here and comment it out to just remind you of the importance of train-test split in modeling training.

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Normalization
Normalization is essential, especially when using SVMs, as it's sensitive to the scale of features. We can use StandardScaler from scikit-learn to normalize the data. It transforms the data so that each feature has a mean of 0 and a standard deviation of 1, which can be important for certain algorithms that are sensitive to the scale of input features, like gradient descent-based optimization methods. Other scalers include Min-Max scaling ("MinMaxScaler"), which scales features to a specific range (typically [0, 1]), and Robust scaling ("RobustScaler"), which is less affected by outliers in the data compared to StandardScaler. These scalers are used to preprocess data and make it more suitable for machine learning algorithms.

It is important to fit the scaler on the training data only and afterwards apply the fitted scaler to the test data. When fitting the scaler on both the train and test data there might be unwanted information leackage.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Grid search
Hyperparameter tuning helps find the best set of hyperparameters for your model. We'll use GridSearchCV from scikit-learn to search for the best combination of parameters. Grid search is a hyperparameter tuning technique in machine learning used to systematically search through a predefined set of hyperparameter combinations for a given model to identify the best-performing configuration. It automates the process of selecting the optimal hyperparameters, helping improve the model's performance and generalization on unseen data by finding the best parameter values that minimize errors or maximize a chosen performance metric.

The GridSearchCV applies for each parameter set a cross validation (in this case five splits). By executing this cross validation we can increase the robustness of the results.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

# Define a parameter grid for hyperparameter tuning
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [ 1, 10],
    'epsilon': [ 1, 10]
}   # specify the settings, try out all combinations of parameters output the best parameter to retrain the model; change the setting in exercise

# Create the GridSearchCV object
grid_search = GridSearchCV(SVR(), param_grid, cv=5, verbose=2)

# Fit the grid search to the scaled training data
grid_search.fit(X_train_scaled, y_train)

# Get the best parameters
best_params = grid_search.best_params_

print("Best Parameters:", best_params)
print("Best Score:", grid_search.best_score_)

## Train a SVM regression model
Now, create an SVM regression model using the best parameters and train it using the training data.

In [ ]:
# Create an SVR model with the best parameters from the grid search
best_svr = SVR(kernel=best_params['kernel'], C=best_params['C'], epsilon=best_params['epsilon'])
best_svr.fit(X_train, y_train)


## Make predictions
Use the trained model to make predictions on the test data and evaluate the performance of the SVM regression model using metrics like Mean Squared Error (MSE) and R-squared.

In [ ]:
y_pred = best_svr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

## Let's see the predictions
we create a "Actual vs. Predicted Values" graph again to give a visual inspection of the prediction quality.

In [ ]:
# y_test contains the actual target values for the test dataset
# y_pred contains the predicted values for the test dataset

# Create a scatter plot to visualize the relationship
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, alpha=0.5)  # Plot actual vs. predicted values

# Add labels and title
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Actual vs. Predicted Values")

# Add a diagonal line for reference (perfect predictions)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', lw=2)

# Show the plot
plt.show()